<a href="https://colab.research.google.com/github/Abhyu001/Osteoporosis-Predection/blob/main/Osteoporosis_Predection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import optuna
from sklearn.ensemble import RandomForestClassifier

In [ ]:
data = pd.read_csv('/content/UA (1).csv')
data.head(5)

,Gender,Age,Height,Weight,BMI,L1-4,L1.4T,FN,FNT,TL,...,Hyperuricemia,AS,VT,VD,OP,CAD,CKD,Fracture,Smoking,Drinking
0,2,61.9,164.0,47.0,17.474717,0.894,-2.4,0.6895,-2.95,0.7130,...,1,1,0,1,1,1,0,0,1,0
1,2,55.0,162.0,54.0,20.576132,1.333,1.3,0.9130,-1.30,1.0675,...,1,1,0,1,1,0,1,0,0,0
2,2,44.0,160.0,54.0,21.093750,1.157,-0.2,0.5190,-3.85,0.5770,...,1,1,0,1,1,1,0,0,0,0
3,1,64.7,158.0,59.0,23.634033,0.948,-2.3,0.7920,-2.15,0.9050,...,1,1,0,1,1,1,0,0,1,1
4,1,88.5,167.0,60.0,21.513859,1.114,-0.9,0.8250,-1.90,0.9385,...,1,1,0,1,1,1,0,0,1,0


In [ ]:
data.isnull().sum()

,0
Gender,0
Age,36
Height,34
Weight,34
BMI,34
L1-4,0
L1.4T,0
FN,0
FNT,0
TL,0


In [ ]:
data = data.fillna(0)

In [ ]:
data.isnull().sum()

,0
Gender,0
Age,0
Height,0
Weight,0
BMI,0
L1-4,0
L1.4T,0
FN,0
FNT,0
TL,0


In [ ]:
x = data.drop(columns=['OP'], axis = 1)
y = data['OP']

pre-processing

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 3)
forest = RandomForestClassifier(n_estimators=100, criterion='entropy',
                             max_features='sqrt',min_samples_leaf=10,random_state=100)

feature selection

filter method

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
selector = SelectKBest(mutual_info_classif, k=3) # Use mutual_info_classif which can handle negative values

x_train_filtered = selector.fit_transform(x_train, y_train)

x_test_filtered = selector.transform(x_test)

# Fit the RandomForestClassifier model before making predictions
forest.fit(x_train_filtered, y_train)  # Add this line to train the model

y_pred = forest.predict(x_test_filtered)

selected_features = selector.get_support(indices=True)
print("Selected feature indices:", selected_features)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the model: {:.2f}".format(accuracy))

Selected feature indices: [ 7  9 10]
Accuracy of the model: 0.82


hybrid method

In [ ]:
'''forest = RandomForestClassifier(n_estimators=100, criterion='entropy',
                             max_features='sqrt',min_samples_leaf=10,random_state=100).fit(x_train, y_train)
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]
top_features_indices = indices[:2]
x_train = x_train[:, top_features_indices]
x_test = x_test[:, top_features_indices]
y_pred = forest.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the model: {:.2f}".format(accuracy))'''

'forest = RandomForestClassifier(n_estimators=100, criterion=\'entropy\',\n                             max_features=\'sqrt\',min_samples_leaf=10,random_state=100).fit(x_train, y_train)\nimportances = forest.feature_importances_\n\nindices = np.argsort(importances)[::-1]\ntop_features_indices = indices[:2]\nx_train = x_train[:, top_features_indices]\nx_test = x_test[:, top_features_indices]\ny_pred = forest.predict(x_test)\naccuracy = accuracy_score(y_test, y_pred)\nprint("Accuracy of the model: {:.2f}".format(accuracy))'

In [ ]:
y.value_counts()

,count
OP,
0,969
1,568


In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier # Import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
forest = RandomForestClassifier(n_estimators=100, criterion='entropy',
                             max_features='sqrt',min_samples_leaf=10,random_state=100)
selector = SelectKBest(mutual_info_classif, k=3)
x_train_filtered = selector.fit_transform(x_train, y_train)
x_test_filtered = selector.transform(x_test)
forest.fit(x_train_filtered, y_train)
y_pred = forest.predict(x_test_filtered)

dimensionality reduction

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 3) # Change number of components to 3
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

y_pred = forest.predict(x_test_pca) # Now the test data also has 3 features
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the model: {:.2f}".format(accuracy))

Accuracy of the model: 0.37


Boosting

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)

params = {
    'objective': 'binary:logistic',
    'max_depth': 4,
    'eta': 0.1,
    'eval_metric': 'logloss'
}

num_rounds = 100

bst = xgb.train(params, dtrain, num_rounds)

y_pred = bst.predict(dtest)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy: {accuracy * 100:.2f}")

Accuracy: 83.77


In [ ]:
!pip install tensorflow

Neural Networks

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu', name = 'Pavani'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

_, accuracy = model.evaluate(x_test, y_test, verbose=0)
y_pred = (model.predict(x_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
print(f"Prediction Accuracy: {accuracy * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 77ms/step - accuracy: 0.6341 - loss: 0.6664 - val_accuracy: 0.7642 - val_loss: 0.5221
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7851 - loss: 0.4868 - val_accuracy: 0.7805 - val_loss: 0.4603
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8353 - loss: 0.4089 - val_accuracy: 0.7886 - val_loss: 0.4391
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8446 - loss: 0.3885 - val_accuracy: 0.7846 - val_loss: 0.4308
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8481 - loss: 0.3690 - val_accuracy: 0.7805 - val_loss: 0.4283
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8610 - loss: 0.3483 - val_accuracy: 0.7846 - val_loss: 0.4269
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8689 - loss: 0.3288 - val_accuracy: 0.7805 - val_loss: 0.4242
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8758 - loss: 0.3271 - val_accuracy: 0.7886 - val_lo

In [ ]:
model.save('my_model.h5')

In [ ]:
loaded_model = tf.keras.models.load_model('my_model.h5')

SVM

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='rbf')
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy : {accuracy*100:.2f}")

Accuracy : 82.79


**Hyperparameter Tuning**

Grid Search

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(estimator=forest, param_grid=param_grid,
                           cv=5, scoring='accuracy', n_jobs=-1).fit(x, y)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best Parameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
Best Score: 0.8184673632556368


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 3)
forest.fit(x_train, y_train)
y_pred = forest.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the model: {:.2f}".format(accuracy))

Accuracy of the model: 0.83


In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

_, accuracy = model.evaluate(x_test, y_test, verbose=0)
y_pred = (model.predict(x_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
print(f"Prediction Accuracy: {accuracy * 100:.2f}%")

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5507 - loss: 0.7203 - val_accuracy: 0.7561 - val_loss: 0.5047
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7841 - loss: 0.4680 - val_accuracy: 0.7764 - val_loss: 0.4449
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8211 - loss: 0.4055 - val_accuracy: 0.7805 - val_loss: 0.4317
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8433 - loss: 0.3674 - val_accuracy: 0.7846 - val_loss: 0.4264
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8445 - loss: 0.3571 - val_accuracy: 0.7805 - val_loss: 0.4249
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8449 - loss: 0.3562 - val_accuracy: 0.7886 - val_loss: 0.4216
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8628 - loss: 0.3251 - val_accuracy: 0.7886 - val_loss: 0.4202
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8558 - loss: 0.3307 - val_accuracy: 0.7927 - val_loss: 0.4224
Epo

Hyperopt

In [ ]:
!pip install hyperopt scikit-learn pandas numpy

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials # Import hp here
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier # Import RandomForestClassifier

def objective(params):
    clf = RandomForestClassifier(
        n_estimators=int(params['n_estimators']),
        max_depth=params['max_depth'] if params['max_depth'] != 'None' else None,
        min_samples_split=int(params['min_samples_split']),
        min_samples_leaf=int(params['min_samples_leaf'])
    )

    score = cross_val_score(clf, x, y, cv=5, scoring='accuracy').mean()

    return {'loss': -score, 'status': STATUS_OK}

space = {
    'n_estimators': hp.choice('n_estimators', [50, 100, 150]),
    'max_depth': hp.choice('max_depth', [None, 10, 20, 30]),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4])
}

trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)
print("Best Hyperparameters:", best)

100%|██████████| 50/50 [02:19<00:00,  2.79s/trial, best loss: -0.8165150810101951]
Best Hyperparameters: {'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 1, 'n_estimators': 2}


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 3)
forest.fit(x_train, y_train)
y_pred = forest.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the model: {:.2f}".format(accuracy))

Accuracy of the model: 0.83


In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

_, accuracy = model.evaluate(x_test, y_test, verbose=0)
y_pred = (model.predict(x_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
print(f"Prediction Accuracy: {accuracy * 100:.2f}%")

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.6134 - loss: 0.6370 - val_accuracy: 0.7642 - val_loss: 0.5036
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8265 - loss: 0.4614 - val_accuracy: 0.7764 - val_loss: 0.4338
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8449 - loss: 0.4122 - val_accuracy: 0.7805 - val_loss: 0.4122
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8448 - loss: 0.3793 - val_accuracy: 0.7927 - val_loss: 0.4101
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8629 - loss: 0.3504 - val_accuracy: 0.8008 - val_loss: 0.4023
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8526 - loss: 0.3632 - val_accuracy: 0.7967 - val_loss: 0.4007
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8658 - loss: 0.3326 - val_accuracy: 0.8049 - val_loss: 0.3996
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8601 - loss: 0.3270 - val_accuracy: 0.8049 - val_loss: 0.3997
